# PETL-AST: Conformer Adapter on ESC-50

**Paper**: Parameter-Efficient Transfer Learning of Audio Spectrogram Transformers (Cappellazzo et al., 2024)

**Runtime**: Go to **Runtime → Change runtime type → T4 GPU** before running.

---

## 1. Setup

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Clone the repo
!git clone https://github.com/yuvalanavi/PETL-AST-Project.git
%cd PETL-AST-Project

In [ ]:
# Install dependencies (torch is already on Colab — only extra packages get installed)
!pip install -r requirements.txt -q

## 2. Download ESC-50 Dataset

In [ ]:
!bash download_data.sh

In [ ]:
# Verify dataset structure
import os
audio_dir = 'data/ESC-50/audio'
meta_file = 'data/ESC-50/meta/esc50.csv'
n_files = len(os.listdir(audio_dir)) if os.path.isdir(audio_dir) else 0
print(f"Audio files: {n_files}")
print(f"Metadata exists: {os.path.isfile(meta_file)}")
assert n_files == 2000, f"Expected 2000 audio files, got {n_files}"
print("Dataset OK ✓")

## 3. Smoke Test (1 fold, 3 epochs)

In [ ]:
!python main.py \
    --data_path 'data' \
    --dataset_name 'ESC-50' \
    --method 'adapter' \
    --adapter_block 'conformer' \
    --adapter_type 'Pfeiffer' \
    --seq_or_par 'parallel' \
    --reduction_rate_adapter 96 \
    --kernel_size 8 \
    --device cuda \
    --num_folds 1 --num_epochs 3 \
    --save_best_ckpt True --output_path './outputs'

**Check**: Loss should decrease over 3 epochs. Trainable params should be ~271K.  
If this works, proceed to full training below.

## 4. Full Training (5 folds × 50 epochs)

**Estimated time**: ~2-3 hours on T4.

In [ ]:
# Clean smoke test outputs before full run
!rm -rf outputs/*

In [ ]:
!python main.py \
    --data_path 'data' \
    --dataset_name 'ESC-50' \
    --method 'adapter' \
    --adapter_block 'conformer' \
    --adapter_type 'Pfeiffer' \
    --seq_or_par 'parallel' \
    --reduction_rate_adapter 96 \
    --kernel_size 8 \
    --device cuda \
    --save_best_ckpt True --output_path './outputs'

## 5. Generate Convergence Plots

In [ ]:
!python utils/visualization.py --log_dir outputs --output_dir outputs/plots

In [ ]:
# Display plots inline
from IPython.display import Image, display
import glob

for img_path in sorted(glob.glob('outputs/plots/*.png')):
    print(img_path)
    display(Image(filename=img_path))

## 6. Evaluate Saved Checkpoints

In [ ]:
!python evaluation.py --data_path data --checkpoint_dir outputs --device cuda

## 7. Download Results

Download checkpoints, logs, and plots to your local machine.

In [ ]:
# Zip all outputs for download
!zip -r outputs_bundle.zip outputs/

from google.colab import files
files.download('outputs_bundle.zip')